In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License")
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an \"AS IS\" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# TensorFlow Hub는 모델을 저장하는 저장소입니다.
# 여기서 이미 학습된 모델을 가져오겠습니다.
import tensorflow_hub as hub

In [ ]:
# beans 데이터셋을 불러오기위해 TFDS를 사용합니다.
# '개발자를 위한 머신러닝&딥러닝' 책에서 TFDS를 사용법을 설명합니다.
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    name = 'beans', 
    split = ['train', 'validation', 'test'],
    as_supervised = True,
    with_info = True)

In [ ]:
# 이제 모델을 학습하기 위해 TFDS의 데이터 처리를 설정합니다.
batch_size=32
def map_data(image, label, target_height = 224, target_width = 224):
    """Normalizes images: `unit8` -> `float32` and resizes images
    by keeping the aspect ratio the same without distortion."""
    image = tf.cast(image, tf.float32)/255.
    image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
    return image, label

ds_train = ds_train.map(map_data)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_validation = ds_validation.map(map_data)
ds_validation = ds_validation.batch(batch_size)
ds_validation = ds_validation.cache()
ds_validation = ds_validation.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(map_data)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# 여러가지 다른 모델로 실험해볼 수 있습니다.
# 아래 주석처리한 것은 카사바 병균(Cassava diseases) 데이터셋에대해 학습된 모델입니다.
# 두번째는 모바일 앱에 주로 사용되는 MobileNet이라는 모델입니다.
#model_handle = "https://tfhub.dev/google/cropnet/feature_vector/cassava_disease_V1/1"
model_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/feature_vector/4"

In [ ]:
# 이제 hub에서 모델을 가져와 'Feature Vector'라고 부르겠습니다.
# 이 레이어를 우리 모델에 추가합니다.
feature_vector = hub.KerasLayer(model_handle, trainable=False,
                               input_shape=(224, 224, 3))

model = tf.keras.models.Sequential([
  feature_vector,
  tf.keras.layers.Dense(3, activation = 'softmax'),
])

In [ ]:
model.summary()

In [ ]:
# 모델을 학습할 때 사용하는 파라미터를 정의하는 곳입니다.
# 손실 함수(loss function)과 최적화기(optimizer)는 학습 방식을 정하게 됩니다.
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(.001),
    metrics = ['accuracy'],
)

In [ ]:
# 모델을 학습할 수 있습니다. Colab의 GPU를 사용하면 20 에폭이 1분 안에 완료됩니다.
num_epochs = 20
history = model.fit(
    ds_train,
    epochs = num_epochs,
    validation_data = ds_validation,
    verbose=1
)

In [ ]:
# 학습된 모델을 saved_model 포맷으로 저장하는데, 나중에는 JS나 TFLite로 변환할 수 있습니다.
tf.saved_model.save(model, '/tmp/saved_model/')

In [ ]:
# 여기는 학습된 모델의 손실과 정확도 그래프를 나타냅니다.
# 정확도는 시간이 지남에 따라 올라가야하고, 손실은 내려가야합니다.
# 여기에는 2개의 그래프가 있습니다. 하나는 학습 데이터에대한 그래프인데, 모델이 이미지를 레이블로 잘 찾아내는지 파악하는데 사용됩니다. 
# 다른 하나는 검증 데이터에 대한 그래프인데, 잘 맞는지를 위한건 아니고, 모델이 한번도 본 적 없는 데이터를 얼마나 잘 맞추는지에대한 지표입니다. 
# 좋은 모델이라면 이 두 곡선이 결국 가까워야합니다.
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# 이 모델은 15 에폭 이후에 정확도가 떨어지는 모습을 보여줍니다.
# 이것은 오버피팅의 신호이므로 좋은 모델이 아닙니다.
# 하지만 지금 예제에서는 이정도로도 충분합니다.

In [ ]:
# 오버피팅은 최종 테스트 정확도에서 입증됩니다.
# 정확도가 92%인데, 테스트 정확도가 89%입니다.
# 이는 모델이 학습하는 동안 '보았던' 데이터에 대해 더 잘 동작한다는 의미이고, 
# 처음보는 데이터는 잘 동작하지 않는다는 의미입니다.

test_loss, test_acc = model.evaluate(ds_test)
print('n Final test accuracy:', test_acc)

In [ ]:
def return_class_labels(ds):
    """`DatasetV1Adapter` object로부터 클래스 레이블 리스트를 반환"""
    l_labels = []
    for _, labels in ds.take(-1):
        labels = labels.numpy()
        l_labels.append(labels[:])
    return [item for sublist in l_labels for item in sublist]

def get_text_label(labelval):
  labels = {
      0: "Angular Leaf Spot",
      1: "Leaf Rust",
      2: "Healthy"
  }
  return labels.get(labelval)

In [ ]:
# 이 코드는 이미지를 플롯으로 그리고, 실제 레이블(진단된 질병)과 예측 레이블(모델이 추론한 질병 결과)을 보여줍니다.
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
normalized_probs = probability_model.predict(ds_test)
predicted_labels = np.argmax(normalized_probs, axis = 1)
actual_labels = return_class_labels(ds_test)

# 테스트 이미지입니다
example = ds_test.take(1)
for sample in example:
    image = sample[0]
    image = image.numpy()

n_cols, n_rows = 4, 4
plt.rcParams['figure.figsize'] = [n_cols*8, n_rows*8]

fig = plt.figure()
for i in range(1, n_cols*n_rows + 1):
    ax = fig.add_subplot(n_rows, n_cols,i)
    ax.text(5, -9, "actual: " + get_text_label(actual_labels[i]) + ", predicted: " + get_text_label(predicted_labels[i]) ,
            color = 'red', fontsize = 15)
    ax.imshow(image[i, :, :, :], cmap = plt.get_cmap("jet"))